# Part 3: Tuning

In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)
pitcher = pd.read_csv('~\Documents\CS\CIS399\greinke_part2_done.csv') 

In [9]:
import os
home_path =  os.path.expanduser('~')
os.chdir(home_path + '\Documents\Github\datascience_1')
!git pull

Already up-to-date.


In [10]:
import sys
sys.path.append(home_path + '\Documents\Github\datascience_1')
from week7 import *
%who function

accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_mean	 compute_prediction	 compute_training	 euclidean_distance	 
f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 informedness	 initialize_centroids	 k_fold	 
k_means	 mulit_gig	 multi_gig	 multi_gini	 multi_probabilities	 phase_1	 phase_2	 pitch_compute_prediction	 predictor_case	 
probabilities	 row_to_vect	 seeder	 tree_predictor	 vote_taker	 


# re-inputting functions... skip

In [7]:
def pitch_compute_prediction(table, target):
    counts = [sum(table['pitch'] == 0),sum(table['pitch'] == 1),sum(table['pitch'] == 2),sum(table['pitch'] == 3)]
    max_val = max(counts)
    if max_val == 0:
        raise LookupError('Prediction impossible - Empty tree on leaf')
    for i in range(len(counts)):
        if counts[i] == max_val:
            return i
        else:
            continue
  

In [8]:
def multi_probabilities(counts):
    count_0 = 0 if 0 not in counts else counts[0]  #could have no 0 values
    count_1 = 0 if 1 not in counts else counts[1]
    count_2 = 0 if 2 not in counts else counts[2]
    count_3 = 0 if 3 not in counts else counts[3]
    total = count_0 + count_1 + count_2 + count_3
    probs = (0,0,0,0) if total == 0 else (1.0*count_0/total, 1.0*count_1/total,1.0*count_2/total,1*0*count_3/total)  #build 2-tuple
    return probs

In [11]:
def multi_gini(counts):
    (p0,p1,p2,p3) = probabilities(counts)
    sum_probs = p0**2 + p1**2 + p2**2 + p3**2
    gini = 1 - sum_probs
    return gini

In [12]:
def gig(starting_table, split_column, target_column):
    

    true_table = starting_table.loc[starting_table[split_column] == 1]
    false_table = starting_table.loc[starting_table[split_column] == 0]
    

    true_counts = true_table[target_column].value_counts()
    false_counts = false_table[target_column].value_counts()  
    starting_counts = starting_table[target_column].value_counts() 
    
    starting_gini = multi_gini(starting_counts)
    true_gini = multi_gini(true_counts)
    false_gini = multi_gini(false_counts)


    starting_size = len(starting_table.index)*1.0
    true_weight = 0.0 if starting_size == 0 else len(true_table.index)/starting_size
    false_weight = 0.0 if starting_size == 0 else len(false_table.index)/starting_size
    

    gig = starting_gini - (true_weight * true_gini + false_weight * false_gini)
    
    return gig

In [78]:
import random
random.seed(1000)
new_seed = seeder(100)
def forest_builder(table, column_choices, target, hypers):
    tree_n = 5 if 'total-trees' not in hypers else hypers['total-trees']
    m = int(len(column_choices)**.5) if 'm' not in hypers else hypers['m']
    k = hypers['max-depth'] if 'max-depth' in hypers else min(2, len(column_choices))
    gig_cutoff = hypers['gig-cutoff'] if 'gig-cutoff' in hypers else 0.0

    #build a single tree - call it multiple times to build multiple trees
    def iterative_build(k):
        train = table.sample(frac=1.0, replace=True, random_state=new_seed())
        train = train.reset_index()
        left_out = table.loc[~table.index.isin(train['index'])]
        left_out = left_out.reset_index() # this gives us the old index in its own column
        oob_list = left_out['index'].tolist()  # list of row indices from original table
        
        rcols = random.sample(column_choices, m)  # subspcace sampling
        columns_sorted = find_best_splitter(train, rcols, target)
        (best_column, gig_value) = columns_sorted[0]

        #Note I add _1 or _0 to make it more readable for debugging
        current_paths = [{'conjunction': [(best_column+'_1', build_pred(best_column, 1))],
                          'prediction': None,
                          'gig_score': gig_value},
                         {'conjunction': [(best_column+'_0', build_pred(best_column, 0))],
                          'prediction': None,
                          'gig_score': gig_value}
                        ]
        k -= 1  # we just built a level as seed so subtract 1 from k
        tree_paths = []  # add completed paths here

        while k>0:
            new_paths = []
            for path in current_paths:
                conjunct = path['conjunction']  # a list of (name, lambda)
                before_table = generate_table(train, conjunct)  #the subtable the current conjunct leads to
                rcols = random.sample(column_choices, m)  # subspace
                columns_sorted = find_best_splitter(before_table, rcols, target)
                (best_column, gig_value) = columns_sorted[0]
                if gig_value > gig_cutoff:
                    new_path_1 = {'conjunction': conjunct + [(best_column+'_1', build_pred(best_column, 1))],
                                'prediction': None,
                                 'gig_score': gig_value}
                    new_paths.append( new_path_1 ) #true
                    new_path_0 = {'conjunction': conjunct + [(best_column+'_0', build_pred(best_column, 0))],
                                'prediction': None,
                                 'gig_score': gig_value
                                 }
                    new_paths.append( new_path_0 ) #false
                else:
                    #not worth splitting so complete the path with a prediction
                    path['prediction'] = pitch_compute_prediction(before_table, target)
                    tree_paths.append(path)
            #end for loop

            current_paths = new_paths
            if current_paths != []:
                k -= 1
            else:
                break  # nothing left to extend so have copied all paths to tree_paths
        #end while loop

        #Generate predictions for all paths that have None
        for path in current_paths:
            conjunct = path['conjunction']
            before_table = generate_table(train, conjunct)
            path['prediction'] = pitch_compute_prediction(before_table, target)
            tree_paths.append(path)
        return (tree_paths, oob_list)
    
    #let's build the forest
    forest = []
    for i in range(tree_n):
        (paths, oob) = iterative_build(k)
        forest.append({'paths': paths, 'weight': None, 'oob': oob})
        
    return forest

SyntaxError: invalid syntax (<ipython-input-78-b2ba6207571f>, line 26)

# Look at the random forest to improve it 

In [82]:
features = ['prior_ball','prior_foul_bunt','prior_foul_tip','prior_hit_by_pitch','prior_hit_into_play',
            'prior_hit_into_play_no_out', 'prior_hit_into_play_score','prior_intent_ball', 'prior_missed_bunt',
            'prior_swinging_strike','prior_swinging_strike_blocked',
            'pk_coors',
            'on_3b','on_2b','on_1b',
            'outs_0','outs_1','outs_2',
            'stand_L','stand_R',
            'strike_0','strike_1','strike_2',
            'ball_0','ball_1','ball_2','ball_3',
            'times_faced_0','times_faced_1','times_faced_2','times_faced_3',
            'pcount_1','pcount_2','pcount_3','pcount_4','pcount_5','pcount_6','pcount_7','pcount_8','pcount_9','pcount_10',
            'pcount_11','pcount_12','pcount_13']

First I want to see if any trees vote unanimously. I will create a matrix of all tree predictions and transpose the matrix to look at each tree and its votes as a row in a dataframe.

In [20]:
pitcher_rf = forest_builder(pitcher, features, 'pitch', {'total-trees':10,'max-depth':15})

In [22]:
all_votes = []
for i in range(len(pitcher)):
    row_predictions = []
    for j in range(10):
        row_predictions.append(tree_predictor(pitcher.loc[i], pitcher_rf[j]))
    all_votes.append(row_predictions)

In [23]:
def unanimous_rows(all_votes):
    rows = []
    all_votes = pd.DataFrame(all_votes)
    for i in range(len(all_votes)):
        row = all_votes.iloc[i]
        case = row[0]
        for j in range(len(row)):
            if row[j] != case:
                break
            else:
                if j == len(row) - 1:
                    rows.append(i)
    return rows


No unanimous trees.

In [42]:
unanimous_rows(pd.DataFrame(all_votes).transpose()) 

[]

In [43]:
all_votes_transpose = pd.DataFrame(all_votes).transpose()

Maybe I can indentify better trees by looking at any trees with a lower amount of fastball predictions.

In [76]:
better_trees = []
for i in range(len(all_votes_transpose)):
    if all_votes_transpose.iloc[i].value_counts()[0] >= len(all_votes_transpose.iloc[i])/2.0:
        continue
    else:
        better_trees.append(i)
    

In [77]:
better_trees

[]

Too many fastball predictions!

In [75]:
for i in range(10):    
    print all_votes_transpose.iloc[i].value_counts()

0    8577
3     275
2      28
1      19
Name: 0, dtype: int64
0    8385
2     246
3     154
1     114
Name: 1, dtype: int64
0    8489
3     271
2      73
1      66
Name: 2, dtype: int64
0    8379
2     388
3     110
1      22
Name: 3, dtype: int64
0    8325
2     525
3      26
1      23
Name: 4, dtype: int64
0    8646
3     178
2      74
1       1
Name: 5, dtype: int64
0    8279
2     279
3     277
1      64
Name: 6, dtype: int64
0    8738
2      71
1      65
3      25
Name: 7, dtype: int64
0    7958
3     503
2     384
1      54
Name: 8, dtype: int64
0    8299
3     360
2     128
1     112
Name: 9, dtype: int64


One thing I appreciated about the decision tree is that it was able to identify batter stance (stand_L) as the best stump. I am going to try to hardcode this into the random forest generator to see if it helps.

In [83]:
import random
random.seed(1000)
new_seed = seeder(100)
def mod_forest_builder(table, column_choices, target, hypers):
    tree_n = 5 if 'total-trees' not in hypers else hypers['total-trees']
    m = int(len(column_choices)**.5) if 'm' not in hypers else hypers['m']
    k = hypers['max-depth'] if 'max-depth' in hypers else min(2, len(column_choices))
    gig_cutoff = hypers['gig-cutoff'] if 'gig-cutoff' in hypers else 0.0

    #build a single tree - call it multiple times to build multiple trees
    def iterative_build(k):
        train = table.sample(frac=1.0, replace=True, random_state=new_seed())
        train = train.reset_index()
        left_out = table.loc[~table.index.isin(train['index'])]
        left_out = left_out.reset_index() # this gives us the old index in its own column
        oob_list = left_out['index'].tolist()  # list of row indices from original table
        
        rcols = random.sample(column_choices, m)  # subspcace sampling
        columns_sorted = find_best_splitter(train, rcols, target)
        (best_column, gig_value) = columns_sorted[0]

        #Note I add _1 or _0 to make it more readable for debugging
        current_paths = [{'conjunction': [('stand_L_1', build_pred('stand_L', 1))],
                          'prediction': None,
                          'gig_score': gig_value},
                         {'conjunction': [('stand_L_0', build_pred('stand_L', 0))],
                          'prediction': None,
                          'gig_score': gig_value}
                        ]
        k -= 1  # we just built a level as seed so subtract 1 from k
        tree_paths = []  # add completed paths here

        while k>0:
            new_paths = []
            for path in current_paths:
                conjunct = path['conjunction']  # a list of (name, lambda)
                before_table = generate_table(train, conjunct)  #the subtable the current conjunct leads to
                rcols = random.sample(column_choices, m)  # subspace
                columns_sorted = find_best_splitter(before_table, rcols, target)
                (best_column, gig_value) = columns_sorted[0]
                if gig_value > gig_cutoff:
                    new_path_1 = {'conjunction': conjunct + [(best_column+'_1', build_pred(best_column, 1))],
                                'prediction': None,
                                 'gig_score': gig_value}
                    new_paths.append( new_path_1 ) #true
                    new_path_0 = {'conjunction': conjunct + [(best_column+'_0', build_pred(best_column, 0))],
                                'prediction': None,
                                 'gig_score': gig_value
                                 }
                    new_paths.append( new_path_0 ) #false
                else:
                    #not worth splitting so complete the path with a prediction
                    path['prediction'] = pitch_compute_prediction(before_table, target)
                    tree_paths.append(path)
            #end for loop

            current_paths = new_paths
            if current_paths != []:
                k -= 1
            else:
                break  # nothing left to extend so have copied all paths to tree_paths
        #end while loop

        #Generate predictions for all paths that have None
        for path in current_paths:
            conjunct = path['conjunction']
            before_table = generate_table(train, conjunct)
            path['prediction'] = pitch_compute_prediction(before_table, target)
            tree_paths.append(path)
        return (tree_paths, oob_list)
    
    #let's build the forest
    forest = []
    for i in range(tree_n):
        (paths, oob) = iterative_build(k)
        forest.append({'paths': paths, 'weight': None, 'oob': oob})
        
    return forest

In [84]:
stand_pitcher_rf = mod_forest_builder(pitcher, features, 'pitch', {'total-trees':10,'max-depth':15})

In [88]:
stand_pitcher_rf[0]['paths']

[{'conjunction': [('stand_L_0', <function week4.<lambda>>),
   ('prior_swinging_strike_1', <function week4.<lambda>>),
   ('pcount_3_1', <function week4.<lambda>>)],
  'gig_score': 0.002266884074291292,
  'prediction': 3},
 {'conjunction': [('stand_L_1', <function week4.<lambda>>),
   ('strike_0_1', <function week4.<lambda>>),
   ('pcount_1_1', <function week4.<lambda>>),
   ('times_faced_2_1', <function week4.<lambda>>)],
  'gig_score': 0.009495466411207465,
  'prediction': 0},
 {'conjunction': [('stand_L_1', <function week4.<lambda>>),
   ('strike_0_0', <function week4.<lambda>>),
   ('pcount_5_1', <function week4.<lambda>>),
   ('prior_foul_tip_1', <function week4.<lambda>>)],
  'gig_score': 0.006936765197121164,
  'prediction': 3},
 {'conjunction': [('stand_L_0', <function week4.<lambda>>),
   ('prior_swinging_strike_1', <function week4.<lambda>>),
   ('pcount_3_0', <function week4.<lambda>>),
   ('pcount_5_1', <function week4.<lambda>>)],
  'gig_score': 0.0015431107332705973,
  'p

In [89]:
all_votes = []
for i in range(len(pitcher)):
    row_predictions = []
    for j in range(10):
        row_predictions.append(tree_predictor(pitcher.loc[i], stand_pitcher_rf[j]))
    all_votes.append(row_predictions)

In [92]:
unanimous_rows(pd.DataFrame(all_votes).transpose()) 

[]

In [90]:
all_votes_transpose = pd.DataFrame(all_votes).transpose()

I have slightly reduced the number of fastballs predicted.

In [91]:
for i in range(10):    
    print all_votes_transpose.iloc[i].value_counts()

0    8386
3     262
2     211
1      40
Name: 0, dtype: int64
0    8289
2     370
3     159
1      81
Name: 1, dtype: int64
0    7897
2     859
3     115
1      28
Name: 2, dtype: int64
0    7925
2     567
1     302
3     105
Name: 3, dtype: int64
0    8235
2     324
3     241
1      99
Name: 4, dtype: int64
0    8270
2     314
3     251
1      64
Name: 5, dtype: int64
0    7710
3     677
2     457
1      55
Name: 6, dtype: int64
0    7746
2     717
3     398
1      38
Name: 7, dtype: int64
0    8129
3     431
2     319
1      20
Name: 8, dtype: int64
0    8212
3     602
2      46
1      39
Name: 9, dtype: int64


Ulitmately, I belive the features I am working with will have more predictive power when looked at wholeistically. This is why I believe it is ultimately not worth the effort to build the random forest for this case. The tree built with max-depth 20 could give a prediction with an accuracy rate of almost 2% better than just guessing fastball everytime. I belive with even more features the model can become even more accurate. This could be another reason why the random forest is under-performing; there are too little features compared to the amount of data points.